In [183]:
%%time
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import math
import xgboost as xgb
from scipy.stats import skew
from scipy import stats
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, auc, classification_report, roc_curve, roc_auc_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from datetime import datetime
from sklearn.neighbors import NearestNeighbors

CPU times: user 66 µs, sys: 1 µs, total: 67 µs
Wall time: 70.3 µs


In [184]:
import re
import string

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# from bs4 import BeautifulSoup
%matplotlib inline

import pickle as pkl

In [185]:
print("XGBoost", xgb.__version__)

XGBoost 0.90


In [186]:
df = pd.read_csv('/content/ks-projects-201801.csv')

In [187]:
df.head()
print(df.shape)

(378661, 15)


In [188]:
df.isnull().sum()

ID                     0
name                   4
category               0
main_category          0
currency               0
deadline               0
goal                   0
launched               0
pledged                0
state                  0
backers                0
country                0
usd pledged         3797
usd_pledged_real       0
usd_goal_real          0
dtype: int64

In [189]:
df["state"].unique()

array(['failed', 'canceled', 'successful', 'live', 'undefined',
       'suspended'], dtype=object)

In [190]:
df['launched'] = pd.to_datetime(df['launched'])
df['deadline'] = pd.to_datetime(df['deadline'])

In [191]:
df['duration'] = df['deadline'] - df['launched']

In [192]:
df.duration

0        58 days 11:47:32
1        59 days 19:16:03
2        44 days 23:39:10
3        29 days 20:35:49
4        55 days 15:24:57
               ...       
378656   29 days 21:24:30
378657   26 days 20:24:46
378658   45 days 04:19:30
378659   30 days 05:46:07
378660   27 days 14:52:13
Name: duration, Length: 378661, dtype: timedelta64[ns]

In [193]:
df.state.value_counts()

failed        197719
successful    133956
canceled       38779
undefined       3562
live            2799
suspended       1846
Name: state, dtype: int64

In [194]:
df.drop(df[(df['state'] == 'undefined')].index,inplace = True)


In [195]:
df['state'].value_counts()

failed        197719
successful    133956
canceled       38779
live            2799
suspended       1846
Name: state, dtype: int64

In [196]:
df["state"].replace({"failed": "Failed", "successful": "Success","canceled": "Failed","live": "Success","suspended": "Failed"}, inplace=True)

In [197]:
df.state.value_counts(normalize = True)*100

Failed     63.541625
Success    36.458375
Name: state, dtype: float64

In [198]:
my_features = ['country', 'currency', 'duration', 'goal']

In [199]:
cols_to_drop = [
        "ID",
        "name",
        "category",
        "main_category",
        "deadline",
        "launched",
        "pledged",
        "backers",
        "usd pledged",
        "usd_pledged_real",
        "usd_goal_real",
    ]
df = df.drop(cols_to_drop, axis=1)

In [200]:
# df.shape

In [201]:
dtrain = df[my_features]
dtest = df[my_features]

In [202]:
df['state'] = df['state'].replace({'failed':0, 'successful':1})

In [203]:
y = df.state

X_train, X_val, y_train, y_val = train_test_split(df, y, test_size=0.1, random_state=0)
categorical_cols = [cname for cname in dtrain.columns if
                    dtrain[cname].nunique() < 10 and 
                    dtrain[cname].dtype == "object"]

numerical_cols = [cname for cname in dtrain.columns if 
                dtrain[cname].dtype in ['int64', 'float64']]

In [204]:
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(337589, 5)
(37510, 5)
(337589,)
(37510,)


In [205]:
numerical_transformer = SimpleImputer(strategy='mean')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

sc = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [206]:
Results = pd.DataFrame({'Model': [], 'Accuracy Score': []})

In [207]:
knnc = KNeighborsClassifier()

knn = Pipeline(steps=[('preprocessor',preprocessor),
                      ('sc',StandardScaler()),
                      ('model', knnc)
])

knn.fit(X_train, y_train)

predsKNN = knn.predict(X_val)


# print('Accuracy:', accuracy_score(y_val, predsKNN))
# print('CR:', classification_report(y_val, predsKNN))
# print('CM:',confusion_matrix(y_val, predsKNN))

# res = pd.DataFrame({'Model': ['KNN'],
#                     'Accuracy Score': [accuracy_score(y_val, predsKNN)]})
# Results = Results.append(res)

In [208]:
print('Accuracy:', accuracy_score(y_val, predsKNN))
print('CR:', classification_report(y_val, predsKNN))
print('CM:',confusion_matrix(y_val, predsKNN))

res = pd.DataFrame({'Model': ['KNN'],
                    'Accuracy Score': [accuracy_score(y_val, predsKNN)]})
Results = Results.append(res)

Accuracy: 0.6095707811250334
CR:               precision    recall  f1-score   support

      Failed       0.66      0.78      0.72     23805
     Success       0.45      0.32      0.37     13705

    accuracy                           0.61     37510
   macro avg       0.56      0.55      0.54     37510
weighted avg       0.59      0.61      0.59     37510

CM: [[18527  5278]
 [ 9367  4338]]


In [209]:
from xgboost import XGBClassifier
model_xgb = XGBClassifier(n_estimators= 500, learning_rate = 0.1, max_depth=5, use_label_encoder=False)

xgb = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model_xgb', model_xgb)
])


xgb.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['goal']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  [])])),
                ('model_xgb',
                 XGBClassifier(max_depth=5, n_estimators=500,
                               use_label_encoder=False))])

In [210]:
predictions = xgb.predict(X_val)
print('Accuracy:', accuracy_score(y_val, predictions))
print('CR:', classification_report(y_val, predictions))
print('CM:',confusion_matrix(y_val, predictions))

res = pd.DataFrame({'Model': ['XGB'],
                    'Accuracy Score': [accuracy_score(y_val, predictions)]})
Results = Results.append(res)

Accuracy: 0.6366568914956011
CR:               precision    recall  f1-score   support

      Failed       0.65      0.92      0.76     23805
     Success       0.51      0.14      0.22     13705

    accuracy                           0.64     37510
   macro avg       0.58      0.53      0.49     37510
weighted avg       0.60      0.64      0.57     37510

CM: [[21917  1888]
 [11741  1964]]


In [211]:
# model = RandomForestClassifier(n_estimators=100, random_state=0,criterion='entropy')


# clf = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('sc',StandardScaler()),
#                       ('model', model)
# ])


# clf.fit(X_train, y_train)


# preds = clf.predict(X_val)
# print('Accuracy:', accuracy_score(y_val, preds))
# print('CR:', classification_report(y_val, preds))
# print('CM:',confusion_matrix(y_val, preds))

# res = pd.DataFrame({'Model': ['RandomForest'],
#                     'Accuracy Score': [accuracy_score(y_val, preds)]})
# Results = Results.append(res)

In [212]:
# with open("model_XGB_k","wb") as f:
#     pkl.dump(model_xgb, f)
# xgb.save_model('xgb.model.kick')

In [213]:
pkl.dump(knn, open('model_XGB', 'wb'))